In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import pickle
import os, re
import nltk
BASE_DIR = '../input/'
LABELED_TRAIN_DF = BASE_DIR + 'labeledTrainData.tsv'
UNLABELED_TRAIN_DF = BASE_DIR + 'unlabeledTrainData.tsv'
TEST_DF = BASE_DIR + 'testData.tsv'
print(os.listdir(BASE_DIR))

['word2vec_model_300dim_40minwords_10context', 'sampleSubmission.csv', 'labeledTrainData.tsv', 'test_submission.csv', 'testData.tsv', 'unlabeledTrainData.tsv']


In [2]:
labeled_train = pd.read_csv(LABELED_TRAIN_DF, header = 0, delimiter = '\t', quoting=3)
unlabeled_train = pd.read_csv(UNLABELED_TRAIN_DF, header = 0, delimiter = '\t', quoting=3)
test = pd.read_csv(TEST_DF, header = 0, delimiter = '\t', quoting=3)
print "Read %d labeled train reviews, %d unlabeled train reviews, " \
          "and %d test reviews" % (labeled_train["review"].size, unlabeled_train["review"].size, test["review"].size)

Read 25000 labeled train reviews, 50000 unlabeled train reviews, and 25000 test reviews


Data leakage

Check if test["sentiment"] is correct

In [3]:
test["sentiment"] = test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)

Credits: Kaggle tutorial

In [4]:
from nltk.corpus import stopwords

def review_to_clean_review(review, remove_numbers=True):
    # Function to convert a document to a clean document,
    # optionally removing numbers.  Returns a string.
    #
    # 1. Remove HTML using lxml parser, ranked best by bs4
    review_text = BeautifulSoup(review, "lxml").get_text()
    #
    # TODO: Clean the text! stemming?
    # https://www.kaggle.com/lystdo/lstm-with-word2vec-embeddings#L92
    # https://www.kaggle.com/currie32/the-importance-of-cleaning-text
    #  
    # 2. Remove non-letters and non-numbers
    review_text = re.sub("[^a-zA-Z0-9]", " ", review_text)
    #
    # 3. Optionally remove numbers
    if remove_numbers:
        review_text = re.sub("[0-9]", " ", review_text)
    else:
        review_text = review_text.replace('0', ' zero ')
        review_text = review_text.replace('1', ' one ')
        review_text = review_text.replace('2', ' two ')
        review_text = review_text.replace('3', ' three ')
        review_text = review_text.replace('4', ' four ')
        review_text = review_text.replace('5', ' five ')
        review_text = review_text.replace('6', ' six ')
        review_text = review_text.replace('7', ' seven ')
        review_text = review_text.replace('8', ' eight ')
        review_text = review_text.replace('9', ' nine ')
    # 6. Return a cleaned string
    return(review_text.lower())

def review_to_wordlist(review, remove_stopwords=False, remove_numbers=True):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    # 1. Clean review, split it into words
    words = review_to_clean_review(review).split()
    #
    # 2. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 6. Return a list of words
    return(words)

def review_to_sentences(review, tokenizer, remove_stopwords=False, remove_numbers=True):
    # Function to split a review into parsed sentences. Returns a
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence, \
                                                        remove_stopwords, remove_numbers))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [5]:
from gensim.models import Word2Vec
EMBEDDING_DIM = 300  # Word vector dimensionality
MIN_WORD_COUNT = 40  # Minimum word count. Kaggle set to 40, to avoid attaching too much importance to individual movie titles.
NUM_THREADS = 4  # Number of threads to run in parallel
CONTEXT = 10  # Context window size
DOWNSAMPLING = 1e-3  # Downsample setting for frequent words
WORD2VEC_MODEL_FILE = BASE_DIR + \
    "word2vec_model_" + \
    str(EMBEDDING_DIM) + "dim_" + \
    str(MIN_WORD_COUNT) + "minwords_" + \
    str(CONTEXT) + "context"

word2vec_model = Word2Vec.load(WORD2VEC_MODEL_FILE)

# If you don't plan to train the model any further, calling
# init_sims will make the model much more memory-efficient.
word2vec_model.init_sims(replace=True)

print("Number of words in the word2vec model vocabulary: %d" % len(word2vec_model.wv.vocab))

Number of words in the word2vec model vocabulary: 16490


Prepare

In [6]:
train_clean_reviews = []
# Getting clean reviews from training set
counter = 0.
for review in labeled_train["review"]:
    clean_review = review_to_clean_review(review, remove_numbers=False)
    train_clean_reviews.append(clean_review)
    if counter % 5000. == 0.:
        print "Train review %d of %d" % (counter, len(labeled_train["review"]))
    counter = counter + 1.

# train_clean_reviews = map(
#     lambda review: review_to_clean_review(review, remove_numbers=False),
#     labeled_train["review"])

test_clean_reviews = []
# Getting clean review from testing set
counter = 0.
for review in test["review"]:
    clean_review = review_to_clean_review(review, remove_numbers=False)
    test_clean_reviews.append(clean_review)
    if counter % 5000. == 0.:
        print "Test review %d of %d" % (counter, len(test["review"]))
    counter = counter + 1.

# test_clean_reviews = map(
#     lambda review: review_to_clean_review(review, remove_numbers=False),
#     test["review"])

all_clean_reviews = train_clean_reviews + test_clean_reviews

Train review 0 of 25000
Train review 5000 of 25000
Train review 10000 of 25000
Train review 15000 of 25000
Train review 20000 of 25000
Test review 0 of 25000
Test review 5000 of 25000
Test review 10000 of 25000
Test review 15000 of 25000
Test review 20000 of 25000


In [7]:
# print(train_clean_reviews[0:2])
# print(test_clean_reviews[0:2])

In [8]:
# We vectorize the text corpus by turning each text into a sequence of integers
# Each integer is the index of a token in the dictionary
from keras.preprocessing.text import Tokenizer

MAX_NUM_WORDS_FOR_KERAS_TOKENIZER = 200000
#
# num_words: the maximum number of words to keep, based on frequency.
keras_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS_FOR_KERAS_TOKENIZER)
#
# fit_on_texts accepts a list of strings, a generator of strings or 
# a list of list of strings. In the last case, it assumes each entry of the lists to be a token.
# Here we provide a list of strings.
keras_tokenizer.fit_on_texts(all_clean_reviews)
word_index = keras_tokenizer.word_index
print('Keras Tokenizer found %s unique tokens' % len(word_index))
#
# texts_to_sequences transforms each text in texts to a sequence of integers.
train_sequences = keras_tokenizer.texts_to_sequences(train_clean_reviews)
test_sequences = keras_tokenizer.texts_to_sequences(test_clean_reviews)

Using TensorFlow backend.


Keras Tokenizer found 101376 unique tokens


In [9]:
# We pad all text sequences to the same length.
# By default zeros are padded at the front.
from keras.preprocessing.sequence import pad_sequences

# Set max length for each review sequence.
MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN = 500

train_pad_sequences = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN)
test_pad_sequences = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN)

In [10]:
# Prepare word embedding matrix

# Choose the smaller number of the two as column length of the matrix
num_words = min(MAX_NUM_WORDS_FOR_KERAS_TOKENIZER, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in word2vec_model.wv.vocab:
        embedding_matrix[i] = word2vec_model.wv.get_vector(word)
# Null word embeddings are words that don't exist in the embedding matrix
# and are therefore represented as zero vectors.
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Null word embeddings: 84920


In [11]:
# Split train_sequences into train and validation. Ratio: 80/20
VALIDATION_SPLIT = 0.2
np.random.seed(1234)

# 
perm = np.random.permutation(len(train_sequences))
index_train = perm[:int(len(train_sequences)*(1-VALIDATION_SPLIT))]
index_val = perm[int(len(train_sequences)*(1-VALIDATION_SPLIT)):]

x_train = train_pad_sequences[index_train]
x_val = train_pad_sequences[index_val]
y_train = labeled_train["sentiment"][index_train].tolist()
y_val = labeled_train["sentiment"][index_val].tolist()

print('Randomly split %d pad sequences for training, %d for validation' % (len(x_train) ,len(x_val)))


Randomly split 20000 pad sequences for training, 5000 for validation


In [12]:
x_test = test_pad_sequences
y_test = test["sentiment"]

## Model Architecture

In [32]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Conv1D, MaxPooling1D ,GlobalMaxPooling1D

In [86]:
def layer1_cnn(dropout=0.2, num_filters=64, kernel_size=2):
    model = Sequential()

    embedding_layer = Embedding(
            num_words,
            EMBEDDING_DIM,
            weights=[embedding_matrix],
            input_length=MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN,
            trainable=False)
    output_layer = Dense(1, activation='sigmoid')
    
    model.add(embedding_layer)
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout))
    model.add(output_layer)
    
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [38]:
def layer3_cnn():
    model = Sequential()
    
    NUM_FILTERS = 64

    embedding_layer = Embedding(
            num_words,
            EMBEDDING_DIM,
            weights=[embedding_matrix],
            input_length=MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN,
            trainable=False)
    output_layer = Dense(1, activation='sigmoid')
    
    model.add(embedding_layer)
    model.add(Conv1D(filters=NUM_FILTERS, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(filters=NUM_FILTERS, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(filters=NUM_FILTERS, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(output_layer)
    
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [60]:
layer1_cnn_model = layer1_cnn()
layer1_cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 300)          30413100  
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 499, 64)           38464     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 33        
Total params: 30,453,677
Trainable params: 40,577
Non-trainable params: 30,413,100
___________________________________________________________

In [71]:
layer1_cnn_model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=[x_val, y_val])

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 5s 231us/step - loss: 0.4500 - acc: 0.7872 - val_loss: 0.3195 - val_acc: 0.8624
Epoch 2/10
20000/20000 [==============================] - 4s 198us/step - loss: 0.2934 - acc: 0.8793 - val_loss: 0.2786 - val_acc: 0.8836
Epoch 3/10
20000/20000 [==============================] - 4s 199us/step - loss: 0.2526 - acc: 0.8974 - val_loss: 0.2656 - val_acc: 0.8850
Epoch 4/10
20000/20000 [==============================] - 4s 199us/step - loss: 0.2233 - acc: 0.9128 - val_loss: 0.2986 - val_acc: 0.8772
Epoch 5/10
20000/20000 [==============================] - 4s 199us/step - loss: 0.1954 - acc: 0.9255 - val_loss: 0.2626 - val_acc: 0.8920
Epoch 6/10
20000/20000 [==============================] - 4s 200us/step - loss: 0.1741 - acc: 0.9349 - val_loss: 0.2602 - val_acc: 0.8950
Epoch 7/10
20000/20000 [==============================] - 4s 198us/step - loss: 0.1498 - acc: 0.9474 - val_loss: 0.2871 - v

In [61]:
layer3_cnn_model = layer3_cnn()
layer3_cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 500, 300)          30413100  
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 499, 64)           38464     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 249, 64)           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 248, 64)           8256      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 124, 64)           0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 123, 64)           8256      
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 64)                0         
__________

In [76]:
layer3_cnn_model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=[x_val, y_val])

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 6s 293us/step - loss: 0.4441 - acc: 0.7788 - val_loss: 0.3346 - val_acc: 0.8584
Epoch 2/10
20000/20000 [==============================] - 5s 249us/step - loss: 0.3161 - acc: 0.8704 - val_loss: 0.3216 - val_acc: 0.8624
Epoch 3/10
20000/20000 [==============================] - 5s 250us/step - loss: 0.2775 - acc: 0.8898 - val_loss: 0.2875 - val_acc: 0.8812
Epoch 4/10
20000/20000 [==============================] - 5s 250us/step - loss: 0.2385 - acc: 0.9085 - val_loss: 0.3132 - val_acc: 0.8710
Epoch 5/10
20000/20000 [==============================] - 5s 248us/step - loss: 0.2161 - acc: 0.9204 - val_loss: 0.2944 - val_acc: 0.8764
Epoch 6/10
20000/20000 [==============================] - 5s 250us/step - loss: 0.1966 - acc: 0.9267 - val_loss: 0.3181 - val_acc: 0.8798
Epoch 7/10
20000/20000 [==============================] - 5s 249us/step - loss: 0.1733 - acc: 0.9365 - val_loss: 0.3043 - v

In [62]:
layer1_cnn_model_dropout5 = layer1_cnn(dropout=0.5)
layer1_cnn_model_dropout5.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=[x_val, y_val])

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 5s 227us/step - loss: 0.4863 - acc: 0.7656 - val_loss: 0.3407 - val_acc: 0.8570
Epoch 2/10
20000/20000 [==============================] - 4s 192us/step - loss: 0.3326 - acc: 0.8642 - val_loss: 0.2911 - val_acc: 0.8784
Epoch 3/10
20000/20000 [==============================] - 4s 193us/step - loss: 0.2863 - acc: 0.8873 - val_loss: 0.2802 - val_acc: 0.8830
Epoch 4/10
20000/20000 [==============================] - 4s 191us/step - loss: 0.2599 - acc: 0.9016 - val_loss: 0.2691 - val_acc: 0.8874
Epoch 5/10
20000/20000 [==============================] - 4s 192us/step - loss: 0.2308 - acc: 0.9134 - val_loss: 0.2573 - val_acc: 0.8942
Epoch 6/10
20000/20000 [==============================] - 4s 192us/step - loss: 0.2117 - acc: 0.9262 - val_loss: 0.2952 - val_acc: 0.8800
Epoch 7/10
20000/20000 [==============================] - 4s 193us/step - loss: 0.1903 - acc: 0.9327 - val_loss: 0.2544 - v

In [80]:
y_test_pred_layer1_cnn = layer1_cnn_model.predict(x_test)
y_test_pred_layer1_cnn_dropout5 = layer1_cnn_model_dropout5.predict(x_test)
y_test_pred_layer3_cnn = layer3_cnn_model.predict(x_test)

In [81]:
def to_binary(predicts):
    return map(lambda predict: 1 if predict > 0.5 else 0, predicts)

In [82]:
y_test_pred_layer1_cnn = to_binary(y_test_pred_layer1_cnn)
y_test_pred_layer1_cnn_dropout5 = to_binary(y_test_pred_layer1_cnn_dropout5)
y_test_pred_layer3_cnn = to_binary(y_test_pred_layer3_cnn)

In [83]:
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

In [84]:
def print_auc_score(model_name, y_test_pred):
    print("The AUC score for %s is : %.4f." % (model_name, roc_auc_score(y_test, y_test_pred)))
    return

In [85]:
print_auc_score("layer1 cnn", y_test_pred_layer1_cnn)
print_auc_score("layer1 cnn dropout 0.5", y_test_pred_layer1_cnn_dropout5)
print_auc_score("layer3 cnn", y_test_pred_layer3_cnn)

The AUC score for layer1 cnn is : 0.8894.
The AUC score for layer1 cnn dropout 0.5 is : 0.8886.
The AUC score for layer3 cnn is : 0.8846.


In [91]:
layer1_cnn_32f_5d = layer1_cnn(dropout=0.5, num_filters=128)
layer1_cnn_32f_5d.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=[x_val, y_val])

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 6s 304us/step - loss: 0.4914 - acc: 0.7683 - val_loss: 0.3345 - val_acc: 0.8604
Epoch 2/10
20000/20000 [==============================] - 5s 250us/step - loss: 0.3266 - acc: 0.8683 - val_loss: 0.2860 - val_acc: 0.8764
Epoch 3/10
20000/20000 [==============================] - 5s 250us/step - loss: 0.2808 - acc: 0.8931 - val_loss: 0.2720 - val_acc: 0.8872
Epoch 4/10
20000/20000 [==============================] - 5s 254us/step - loss: 0.2426 - acc: 0.9090 - val_loss: 0.2555 - val_acc: 0.8948
Epoch 5/10
20000/20000 [==============================] - 5s 255us/step - loss: 0.2142 - acc: 0.9229 - val_loss: 0.2702 - val_acc: 0.8878
Epoch 6/10
20000/20000 [==============================] - 5s 255us/step - loss: 0.1874 - acc: 0.9355 - val_loss: 0.2639 - val_acc: 0.8960
Epoch 7/10
20000/20000 [==============================] - 5s 254us/step - loss: 0.1664 - acc: 0.9438 - val_loss: 0.2633 - v

In [103]:
def layer1_cnn_d(dropout=0.2, num_filters=64, kernel_size=2):
    model = Sequential()

    embedding_layer = Embedding(
            num_words,
            EMBEDDING_DIM,
            weights=[embedding_matrix],
            input_length=MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN,
            trainable=False)
    output_layer = Dense(1, activation='sigmoid')
    
    model.add(embedding_layer)
#     model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, padding='valid', activation='relu', strides=1))
#     model.add(MaxPooling1D(2))
#     model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, padding='valid', activation='relu', strides=1))
#     model.add(MaxPooling1D(2))
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(dropout))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout))
    model.add(output_layer)
    
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [104]:
layer1_cnn_d_0 = layer1_cnn_d()
layer1_cnn_d_0.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=[x_val, y_val])

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 5s 275us/step - loss: 0.4981 - acc: 0.7510 - val_loss: 0.3383 - val_acc: 0.8588
Epoch 2/10
20000/20000 [==============================] - 4s 197us/step - loss: 0.3588 - acc: 0.8442 - val_loss: 0.3022 - val_acc: 0.8722
Epoch 3/10
20000/20000 [==============================] - 4s 197us/step - loss: 0.3208 - acc: 0.8652 - val_loss: 0.2831 - val_acc: 0.8820
Epoch 4/10
20000/20000 [==============================] - 4s 197us/step - loss: 0.2920 - acc: 0.8804 - val_loss: 0.2886 - val_acc: 0.8758
Epoch 5/10
20000/20000 [==============================] - 4s 197us/step - loss: 0.2657 - acc: 0.8931 - val_loss: 0.2661 - val_acc: 0.8924
Epoch 6/10
20000/20000 [==============================] - 4s 196us/step - loss: 0.2470 - acc: 0.9011 - val_loss: 0.2576 - val_acc: 0.8922
Epoch 7/10
20000/20000 [==============================] - 4s 196us/step - loss: 0.2254 - acc: 0.9121 - val_loss: 0.2534 - v

In [106]:
y_test_pred_layer1_cnn_d_0 = layer1_cnn_d_0.predict(x_test)
y_test_pred_layer1_cnn_d_0 = to_binary(y_test_pred_layer1_cnn_d_0)
print_auc_score("layer1 cnn_d", y_test_pred_layer1_cnn_d_0)

The AUC score for layer1 cnn_d is : 0.8940.


In [109]:
layer1_cnn_d_5d = layer1_cnn_d(dropout=0.5)
layer1_cnn_d_5d.fit(x_train, y_train, batch_size=64, epochs=20, validation_data=[x_val, y_val])

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 6s 292us/step - loss: 0.6255 - acc: 0.6316 - val_loss: 0.4245 - val_acc: 0.8384
Epoch 2/20
20000/20000 [==============================] - 4s 203us/step - loss: 0.4595 - acc: 0.7944 - val_loss: 0.3529 - val_acc: 0.8516
Epoch 3/20
20000/20000 [==============================] - 4s 202us/step - loss: 0.4015 - acc: 0.8280 - val_loss: 0.3238 - val_acc: 0.8664
Epoch 4/20
20000/20000 [==============================] - 4s 201us/step - loss: 0.3607 - acc: 0.8513 - val_loss: 0.3026 - val_acc: 0.8720
Epoch 5/20
20000/20000 [==============================] - 4s 198us/step - loss: 0.3411 - acc: 0.8593 - val_loss: 0.2909 - val_acc: 0.8772
Epoch 6/20
20000/20000 [==============================] - 4s 202us/step - loss: 0.3191 - acc: 0.8722 - val_loss: 0.2860 - val_acc: 0.8786
Epoch 7/20
20000/20000 [==============================] - 4s 204us/step - loss: 0.3105 - acc: 0.8752 - val_loss: 0.2756 - v

In [110]:
y_test_pred_layer1_cnn_d_5d = layer1_cnn_d_5d.predict(x_test)
y_test_pred_layer1_cnn_d_5d = to_binary(y_test_pred_layer1_cnn_d_5d)
print_auc_score("layer1 cnn_d", y_test_pred_layer1_cnn_d_5d)

The AUC score for layer1 cnn_d is : 0.8932.


In [114]:
def layer3_cnn_1p(dropout=0.2):
    model = Sequential()
    
    NUM_FILTERS = 64

    embedding_layer = Embedding(
            num_words,
            EMBEDDING_DIM,
            weights=[embedding_matrix],
            input_length=MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN,
            trainable=False)
    output_layer = Dense(1, activation='sigmoid')
    
    model.add(embedding_layer)
    model.add(Conv1D(filters=NUM_FILTERS, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(Conv1D(filters=NUM_FILTERS, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(Conv1D(filters=NUM_FILTERS, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout))
    model.add(output_layer)
    
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [116]:
layer3_cnn_1p_0 = layer3_cnn_1p(dropout=0.5)
layer3_cnn_1p_0.fit(x_train, y_train, batch_size=64, epochs=20, validation_data=[x_val, y_val])

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 8s 424us/step - loss: 0.5101 - acc: 0.7393 - val_loss: 0.3379 - val_acc: 0.8498
Epoch 2/20
20000/20000 [==============================] - 6s 314us/step - loss: 0.3480 - acc: 0.8579 - val_loss: 0.3009 - val_acc: 0.8748
Epoch 3/20
20000/20000 [==============================] - 6s 317us/step - loss: 0.2939 - acc: 0.8848 - val_loss: 0.2843 - val_acc: 0.8838
Epoch 4/20
20000/20000 [==============================] - 6s 312us/step - loss: 0.2659 - acc: 0.8964 - val_loss: 0.2789 - val_acc: 0.8824
Epoch 5/20
20000/20000 [==============================] - 6s 309us/step - loss: 0.2405 - acc: 0.9105 - val_loss: 0.2822 - val_acc: 0.8846
Epoch 6/20
20000/20000 [==============================] - 6s 316us/step - loss: 0.2179 - acc: 0.9195 - val_loss: 0.2814 - val_acc: 0.8880
Epoch 7/20
20000/20000 [==============================] - 6s 316us/step - loss: 0.2013 - acc: 0.9263 - val_loss: 0.2721 - v

KeyboardInterrupt: 